In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from sklearn.model_selection import LeaveOneOut
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
#https://keras.io/api/applications/

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

#freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [3]:
def create_model(base_model):
    
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.1)(x)
    x = Dense(100, activation='relu')(x)
    #x = keras.layers.Dropout(0.3)(x)
    
    
    # let's add a fully-connected layer
    x = Dense(231, activation='softmax')(x)
    #x = keras.layers.Dropout(0.3)(x)
    
    # and a logistic layer
    #predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x)
    
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model

In [4]:
def read_images(N, num_classes):

    #Declaring array of images 
    images = []

    #loop to import images from file directory
    for i in range (12): 
        x = np.asarray(Image.open("C:/Users/Mark Fahy/Desktop/Final Year/Final Year Project/bongard-problems-master/bongard-problems-master/p"+str(N)+"/"+str(i)+".png").convert('L'))
        x = np.array((x, x, x))
        #print(x.shape)
        images.append(x)
        
    #np.tile(images, 3)
    #Converting to numpy array
    dataset = np.array(images)
    dataset = dataset.reshape((-1, 104, 104, 3))    

    #Y data
    yvalues = (0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1)
    
    # Scale images to the [0, 1] range
    dataset = dataset.astype("float32") / 255
    

    # convert class vectors to binary class matrices
    #yvalues = keras.utils.to_categorical(yvalues, num_classes)    
    
    return dataset, yvalues

In [5]:
accuracies = []
losses = []
folds = 0
problems = 0

global_x_train = []
global_x_test = []
global_y_train = []
global_y_test = []

num_classes = 231
#input_shape = (104, 104, 1)

for i in range (1, 232): 
    dataset, yvalues = read_images(i, num_classes)
    #problems = problems + 1
    #print("Problem number: ", problems)
    
    x_train, x_test, y_train, y_test = train_test_split(dataset, yvalues, test_size=0.16666)  
    y_train = np.ones_like(y_train) * (i - 1)
    y_test = np.ones_like(y_test) * (i - 1)

    
    global_x_train.extend(x_train)
    global_x_test.extend(x_test)
    global_y_train.extend(y_train)
    global_y_test.extend(y_test)
    
global_x_train = np.array(global_x_train)
global_x_test = np.array(global_x_test)
global_y_train = np.array(global_y_train)
global_y_test = np.array(global_y_test)


# convert class vectors to binary class matrices
global_y_test = keras.utils.to_categorical(global_y_test, num_classes) 
global_y_train = keras.utils.to_categorical(global_y_train, num_classes) 

print(global_y_test.shape)
print(global_y_test)

(462, 231)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [7]:
model = create_model(base_model)   
    
model.fit(global_x_train, global_y_train, epochs = 100, validation_data = (global_x_test, global_y_test))
score = model.evaluate(global_x_test, global_y_test, verbose=0)

print('Accuracy: ', score[1])
print('Loss: ', score[0])
    

#print("Average Accuracy: ", (sum(accuracies) / len(accuracies)) )
#print("Average Loss: ", (sum(losses) / len(losses)) )

Epoch 1/100
73/73 [==============================] - 19s 210ms/step - loss: 5.4484 - accuracy: 0.0121 - val_loss: 5.4013 - val_accuracy: 0.0130
Epoch 2/100
73/73 [==============================] - 20s 276ms/step - loss: 5.0437 - accuracy: 0.0481 - val_loss: 4.9557 - val_accuracy: 0.0541
Epoch 3/100
73/73 [==============================] - 20s 280ms/step - loss: 4.3920 - accuracy: 0.1182 - val_loss: 4.5342 - val_accuracy: 0.0866
Epoch 4/100
73/73 [==============================] - 21s 283ms/step - loss: 3.8004 - accuracy: 0.1827 - val_loss: 4.3146 - val_accuracy: 0.1126
Epoch 5/100
73/73 [==============================] - 21s 287ms/step - loss: 3.3252 - accuracy: 0.2619 - val_loss: 4.2809 - val_accuracy: 0.1255
Epoch 6/100
73/73 [==============================] - 21s 293ms/step - loss: 2.9461 - accuracy: 0.3182 - val_loss: 4.2344 - val_accuracy: 0.1299
Epoch 7/100
73/73 [==============================] - 22s 301ms/step - loss: 2.6431 - accuracy: 0.3645 - val_loss: 4.1879 - val_accuracy:

73/73 [==============================] - 24s 326ms/step - loss: 0.0408 - accuracy: 0.9905 - val_loss: 8.3107 - val_accuracy: 0.1883
Epoch 58/100
73/73 [==============================] - 24s 326ms/step - loss: 0.0562 - accuracy: 0.9879 - val_loss: 7.8542 - val_accuracy: 0.1926
Epoch 59/100
73/73 [==============================] - 24s 332ms/step - loss: 0.0457 - accuracy: 0.9879 - val_loss: 8.2442 - val_accuracy: 0.2143
Epoch 60/100
73/73 [==============================] - 25s 346ms/step - loss: 0.0463 - accuracy: 0.9883 - val_loss: 8.5110 - val_accuracy: 0.1991
Epoch 61/100
73/73 [==============================] - 25s 345ms/step - loss: 0.0458 - accuracy: 0.9909 - val_loss: 8.4789 - val_accuracy: 0.2100
Epoch 62/100
73/73 [==============================] - 26s 352ms/step - loss: 0.0450 - accuracy: 0.9861 - val_loss: 8.5257 - val_accuracy: 0.1861
Epoch 63/100
73/73 [==============================] - 24s 325ms/step - loss: 0.0390 - accuracy: 0.9905 - val_loss: 8.5787 - val_accuracy: 0.203

From running 100 epochs above, the trend shows that anything more than 25-30 epochs is a repitive pattern where the valdtaion accuracy floats between ~0.22 and ~0.18.